In [1]:
import pandas as pd
import numpy as np
from math import *
from src.display import horizontal_cluster
from src.find_HC import *
#from src.Project_step_1 import *

In [2]:
horizontal_cluster('AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG')

[['-' 'A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M']
 ['-' 'I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K']
 ['-' 'Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E']
 ['-' '☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁']
 ['A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M' '-']
 ['I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K' '-']
 ['Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E' '-']
 ['☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁' '-']]


In [3]:
# ça durera 1300 fois plus longtemps pour toute la database
Q_codes, all_HC = get_analyse('part_of_data.txt')

In [4]:
all_HC

{3: 33419,
 7: 8560,
 5: 23365,
 15: 3333,
 21: 7666,
 337: 276,
 675: 51,
 1351: 33,
 81: 1908,
 163: 787,
 327: 93,
 17: 9888,
 35: 3301,
 71: 299,
 11: 7852,
 273: 1347,
 9: 16317,
 73: 2773,
 293: 308,
 37: 1085,
 649: 582,
 137: 3314,
 41: 1867,
 657: 127,
 145: 2756,
 19: 4005,
 325: 200,
 1301: 20,
 69: 994,
 277: 129,
 585: 292,
 113: 209,
 49: 2775,
 13: 8009,
 329: 104,
 659: 20,
 147: 320,
 57: 572,
 25: 5362,
 587: 26,
 75: 200,
 89: 3012,
 713: 149,
 201: 739,
 241: 68,
 31: 462,
 651: 33,
 139: 218,
 29: 2890,
 117: 386,
 53: 2165,
 209: 1207,
 419: 672,
 839: 23,
 13425: 2,
 5233: 3,
 1137: 8,
 27: 3732,
 121: 169,
 1297: 73,
 83: 1176,
 665: 59,
 153: 1219,
 85: 3995,
 341: 2403,
 1173: 244,
 149: 451,
 1679: 7,
 655: 43,
 143: 154,
 229: 37,
 101: 392,
 23: 1649,
 1093: 68,
 115: 160,
 51: 968,
 105: 151,
 841: 19,
 47: 1286,
 333: 37,
 77: 618,
 1299: 391,
 275: 634,
 55: 225,
 357: 125,
 1427: 30,
 403: 140,
 1317: 8,
 227: 64,
 455: 15,
 911: 5,
 99: 1045,
 199: 58,

In [107]:
Q_codes[-1][len(Q_codes[-1])-100:]

[['DVU', 'MVM', 'MUVDVVMDD', 'MVMVVVMUMU', 'VU', 'VDVMVVUU'],
 ['D', 'VVVVM', 'D', 'V', 'UVD', 'V', 'VDUV', 'MDDVMVU'],
 ['MVMMUU', 'V', 'MU'],
 ['VMV', 'VVUUDVUUMVM', 'MM', 'MM'],
 ['D', 'M', 'VU', 'MVM', 'DVV', 'DUV', 'M', 'VM'],
 ['UU', 'VM', 'D', 'VVVUDV', 'MV', 'UVVUVDVV', 'VDUV', 'UDDVUMV'],
 ['MVUDU', 'VUM', 'V', 'DVUUV'],
 ['UUMVMVDDMVDM', 'VV', 'VDU'],
 ['DVM', 'MU', 'MV', 'MVMV', 'V'],
 ['VVU', 'V', 'M', 'UUVV', 'UDMVVVV', 'VM', 'VV'],
 ['UVU', 'VV', 'VU', 'MVMVDUUMDVV', 'UV', 'VUUVDU'],
 ['VVU', 'DV', 'MVV', 'MVV', 'DM', 'MU'],
 ['UV', 'VVVM', 'UV', 'MV', 'UVVUVDVV', 'VDUV', 'DVMVDU'],
 ['V', 'UDUMVU', 'DDUMVVD', 'M', 'VV', 'V', 'UV', 'UV'],
 ['DUUV', 'V', 'MVM', 'MV', 'VVUD', 'DVMVDD'],
 ['VUUMVUDUDDVV', 'V', 'VDU'],
 ['DDMVDDDV', 'MV', 'DV'],
 ['VV', 'D', 'VV', 'UVMDDM', 'MV', 'VDU', 'UVUUVDMVV'],
 ['VVU', 'MV', 'M', 'DVUMVV', 'DV', 'MVMV', 'D'],
 ['D', 'VMU', 'D', 'MV', 'UVD', 'M', 'VDUV', 'DDVUU'],
 ['MVM', 'UMMVDM', 'UM', 'UUMMVDUV'],
 ['DM', 'DVUMVVU', 'DVMDMVVV', 'UV'

In [113]:
#a='AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG'
a='-----....--...----.------...............---...---.---............-----....--.-....-.-..-.--.......--..-...-.-..-.-..-.-.-........................--.-.-...-.--...GQM............PRL..E..VV.Q....T..L.T..A.G...Y.....E....D....V....-L....P....L.L...P.D..........Q.....-....VT.......L............C......N.......A....A.......G.....L......H........D.....T...S....TA..ELAvalalasgrhldwfardqvtgtwgqrpgraladqrvlivgygrigaaiearlqgfevasvtrvsgrgrsgpplvhpvgdlhgllpe..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................advvfvvaphtpqtegmfgarelgllpdgallvnvargklvdtaallaelsarritaaldvtdpeplpvghplwhapgvlisphvgGA..SSAFT.P..R.A.D.RLI.AAQLT.R.FA.AG......Q.PLANVV-'
struct=binary_coding(a)[0]
print(struct)

⯁QM★RLEVVQ☐L☐A⯁YEDVL★LL★DQV☐LCNAA⯁LHD☐▣☐AELAvalalasgrhldwfardqvtgtwgqrpgraladqrvlivgygrigaaiearlqgfevasvtrvsgrgrsgpplvhpvgdlhgllpeadvvfvvaphtpqtegmfgarelgllpdgallvnvargklvdtaallaelsarritaaldvtdpeplpvghplwhapgvlisphvg⯁A▣▣AF☐★RADRLIAAQL☐RFAA⯁Q★LANVV
0010010110010001001101100010100000100000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000001100010010000010011


In [114]:
int('101100100010011', 2)

22803